# Oil type by origin
Goal: Determine oil type weighting in tank traffic by destination and vessel type


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yaml
from monte_carlo_utils import decimal_divide

In [2]:
# User inputs
file_dir  = '/Users/rmueller/Data/MIDOSS/DeptOfEcology/'
file_name = 'MuellerTrans4-30-20.xlsx'

# This is the precision used to calculate oil type weights 
# Trial and error showed that a precision of 2 is neccessary for weights
# to sum to 1.0
precision = 2

In [3]:
# Import columns are: (G) Deliverer, (H) Receiver, (O) Region, (P) Product, 
#                     (Q) Quantity in Gallons, (R) Transfer Type (oiling, Cargo, or Other)', 
#                     (w) DelivererTypeDescription, (x) ReceiverTypeDescription 
df = pd.read_excel(f'{file_dir}{file_name}',sheet_name='Vessel Oil Transfer', 
                   usecols="G,H,P,Q,R,W,X")
# housekeeping
df['Receiver'] = df['Receiver'].str.replace('US Oil Tacoma ', 'U.S. Oil & Refining')
df['Receiver'] = df['Receiver'].str.replace('TLP', 'TLP Management Services LLC (TMS)')
df['Deliverer'] = df['Deliverer'].str.replace('US Oil Tacoma ', 'U.S. Oil & Refining')
df['Deliverer'] = df['Deliverer'].str.replace('TLP', 'TLP Management Services LLC (TMS)')

# create a dataframe for tug traffic from westridge
tugs_from_westridge = pd.DataFrame(columns = df.columns)

## Create a list of all terminals included in the dataset, together with Region 
(so we can isolate Salish Sea) 

In [4]:
# The following list includes facilities used in Casey's origin/destination analysis 
# with names matching the Dept. of Ecology (DOE) database.  
# For example, the shapefile "Maxum Petroleum - Harbor Island Terminal" is labeled as 'Maxum (Rainer Petroleum)'
# in the DOE database.  I use the Ecology language here and will need to translate to Shapefile speak 
facility_names = [ 'BP Cherry Point Refinery', 'Shell Puget Sound Refinery', 'Shell Oil LP Seattle Distribution Terminal',
                   'Maxum (Rainer Petroleum)', 'Tidewater Snake River Terminal','Nustar Energy Tacoma', 
                   'SeaPort Sound Terminal', 'Tesoro Vancouver Terminal','Phillips 66 Ferndale Refinery', 
                   'Phillips 66 Tacoma Terminal', 'Andeavor Anacortes Refinery (formerly Tesoro)',
                   'Tesoro Port Angeles Terminal','U.S. Oil & Refining','Naval Air Station Whidbey Island (NASWI)',
                   'NAVSUP Manchester', 'Alon Asphalt Company (Paramount Petroleum)', 'Kinder Morgan Liquids Terminal - Harbor Island',
                   'Nustar Energy Tacoma', 'Tesoro Pasco Terminal', 'REG Grays Harbor, LLC', 'Tidewater Vancouver Terminal',
                   'TLP Management Services LLC (TMS)']

# define vessel and oil types to include in .yaml file
vessel_types = ['atb', 'barge', 'tanker']
oil_types    = ['akns', 'bunker', 'dilbit', 'jet', 'diesel', 'gas', 'other']
data_types    = ['total_gallons', 'fraction_of_total', 'number_of_transfers']

## Create name lists for the DOE oil types used in our monte-carlo oil categories

In [5]:
# identify all names of oils in DOE database that are attributed to our oil types
oil_classification = {}
for oil in oil_types:
    oil_classification[oil] = []

[nrows,ncols] = df.shape
for row in range(nrows):
    if 'CRUDE' in df.Product[row] and df.Product[row] not in oil_classification['akns']:
        oil_classification['akns'].append(df.Product[row])
    elif 'BAKKEN' in df.Product[row] and df.Product[row] not in oil_classification['akns']:
        oil_classification['akns'].append(df.Product[row])
    elif 'BUNKER' in df.Product[row] and df.Product[row] not in oil_classification['bunker']:
        oil_classification['bunker'].append(df.Product[row])
    elif 'BITUMEN' in df.Product[row] and df.Product[row] not in oil_classification['dilbit']:
        oil_classification['dilbit'].append(df.Product[row])
    elif 'DIESEL' in df.Product[row] and df.Product[row] not in oil_classification['diesel']:
        oil_classification['diesel'].append(df.Product[row])
    elif 'GASOLINE' in df.Product[row] and df.Product[row] not in oil_classification['gas']:
        oil_classification['gas'].append(df.Product[row])
    elif 'JET' in df.Product[row] and df.Product[row] not in oil_classification['jet']:
        oil_classification['jet'].append(df.Product[row])
    elif ('CRUDE' not in df.Product[row] and
          'BAKKEN' not in df.Product[row] and
          'BUNKER' not in df.Product[row] and
          'BITUMEN' not in df.Product[row] and
          'DIESEL' not in df.Product[row] and
          'GASOLINE' not in df.Product[row] and
          'JET' not in df.Product[row] and
          df.Product[row] not in oil_classification['other']):
        oil_classification['other'].append(df.Product[row])

oil_classification

{'akns': ['CRUDE OIL', 'BAKKEN'],
 'bunker': ['BUNKER OIL/HFO'],
 'dilbit': ['BITUMEN OIL (DIL OR SYN)'],
 'jet': ['JET FUEL/KEROSENE'],
 'diesel': ['DIESEL/MARINE GAS OIL', 'BIODIESEL', 'DIESEL LOW SULPHUR (ULSD)'],
 'gas': ['GASOLINE'],
 'other': ['ETHANOL',
  'LUBE OIL/MOTOR OIL',
  'Cat Feed/VGO',
  'CUTTER STOCK',
  'OILY WATER MIXTURE',
  'WASTE OIL',
  'Z-OTHER',
  'HYDRAULIC OIL',
  'OILY WASTE',
  'USED OIL',
  'NAPTHA',
  'EDIBLE/VEGETABLE OIL',
  'DECANT OIL',
  'NONENE',
  'ASPHALT/CREOSEOTE']}

## Sort imports and exports by ship types.  We have three categories: tanker, atb, barge.
1. `tanker`: Listed in DOE dataset as `TANK SHIP` TypeDescription 
2. `atb`: Listed in DOE dataset as `TANK BARGE` or `TUGBOAT` TypeDescription and `ATB` or `ITB` as Deliverer/Receiver
3. `barge`: Listed in DOE dataset as `TANK BARGE` or `TUGBOAT` TypeDescription (no `ATB` or `ITB` in Deliverer/Receiver)

Note-to-self on creating subsets of the `df` dataset to use for tally: If I wanted to only extract the  information in `TransferQtyInGallon`, I'd use
```
cargo_import_df = df.loc[ 
     (df.TransferType == 'Cargo') & 
     (df.Receiver.isin(facility_names)), ['TransferQtyInGallon']
 ].copy(deep = True)
```


In [6]:
## Tally up imports by ship type and marine terminal
tanker_import_total = {}
atb_import_total = {}
barge_import_total = {}
for vessel in vessel_types:
    if vessel =='tanker':
        for facility in facility_names:
            tanker_import_total[facility] = df.loc[
            (df.Receiver == facility) &
            (df.TransferType == 'Cargo') &
            (df.DelivererTypeDescription.isin(['TANK SHIP'])), 
            'TransferQtyInGallon'
        ].sum(axis=0).item()
    elif vessel == 'atb':
        for facility in facility_names:
            atb_import_total[facility] = df.loc[
            (df.Receiver == facility) &
            (df.TransferType == 'Cargo') &
            (df.DelivererTypeDescription.isin(['TANK BARGE','TUGBOAT'])) & 
            ('ATB' in df.Deliverer) | ('ITB' in df.Deliverer), 
            'TransferQtyInGallon'
        ].sum(axis=0).item()
    elif vessel == 'barge':
        for facility in facility_names:
            barge_import_total[facility] = df.loc[
            (df.Receiver == facility) &
            (df.TransferType == 'Cargo') &
            (df.DelivererTypeDescription.isin(['TANK BARGE','TUGBOAT'])) & 
            ('ATB' not in df.Deliverer) & ('ITB' not in df.Deliverer), 
            'TransferQtyInGallon'
        ].sum(axis=0).item()

In [7]:
## Tally up exports by ship type and marine terminal
tanker_export_total = {}
atb_export_total = {}
barge_export_total = {}
for vessel in vessel_types:
    if vessel =='tanker':
        for facility in facility_names:
            tanker_export_total[facility] = df.loc[
            (df.Deliverer == facility) &
            (df.TransferType == 'Cargo') &
            (df.ReceiverTypeDescription.isin(['TANK SHIP'])), 
            'TransferQtyInGallon'
        ].sum(axis=0).item()
    elif vessel == 'atb':
        for facility in facility_names:
            atb_export_total[facility] = df.loc[
            (df.Deliverer == facility) &
            (df.TransferType == 'Cargo') &
            (df.ReceiverTypeDescription.isin(['TANK BARGE','TUGBOAT'])) & 
            ('ATB' in df.Receiver) | ('ITB' in df.Receiver), 
            'TransferQtyInGallon'
        ].sum(axis=0).item()
    elif vessel == 'barge':
        for facility in facility_names:
            barge_export_total[facility] = df.loc[
            (df.Deliverer == facility) &
            (df.TransferType == 'Cargo') &
            (df.ReceiverTypeDescription.isin(['TANK BARGE','TUGBOAT'])) & 
            ('ATB' not in df.Receiver) & ('ITB' not in df.Receiver), 
            'TransferQtyInGallon'
        ].sum(axis=0).item()

## Tally up `total_gallons`, `number_of_transfers`, `fraction_of_total`, where:
- `total_gallons`: Total gallons transfered for the particular oil type
- `fraction_of_total`: (`total_gallons` by oil type) / (total gallons across oil types)
- `number_of_transfers`: Total transfers for the particular oil type

In [8]:

# Create a destination dictionary
destination = {}
destination['names'] = facility_names.copy()

for facility in facility_names:    
    destination[facility] = {}
    for vessel in vessel_types:
        destination[facility][vessel] = {}
        for oil in oil_types:
            destination[facility][vessel][oil] = {}
            for data in data_types:
                destination[facility][vessel][oil][data] = 0

# Create an origin dictionary
origin = {}
origin['names'] = facility_names.copy()  

for facility in facility_names:    
    origin[facility] = {}
    for vessel in vessel_types:
        origin[facility][vessel] = {}
        for oil in oil_types:
            origin[facility][vessel][oil] = {}
            for data in data_types:
                origin[facility][vessel][oil][data] = 0

### tanker imports and exports

In [9]:
vessel = 'tanker'
type_description = ['TANK SHIP']

In [10]:
# ~~~  IMPORTS ~~~

# the following lines select all cargo entries registered as delivered by `tank ships` 
# to the designated marine terminal for the type of oil listed in our 
# `oil_classification` as the DOE oil type represented by our oil categories
for facility in facility_names:
    for oil in oil_types:
        
        # Add up net oil quantity transferred by oil_types
        destination[facility][vessel][oil]['total_gallons'] = df.loc[        
            (df.Receiver == facility) &
            (df.TransferType == 'Cargo') &
            (df.DelivererTypeDescription.isin(type_description)) &
            (df.Product.isin(oil_classification[oil])),
            'TransferQtyInGallon'
        ].sum(axis=0).item()
        
        # Calculate the fraction of total oil transfer represented 
        # by this oil type
        if tanker_import_total[facility] != 0:
            destination[facility][vessel][oil]['fraction_of_total'] = decimal_divide(
                destination[facility][vessel][oil]['total_gallons'], 
                tanker_import_total[facility],
                precision
            )
        
        # Catalogue the number of transfers 
        destination[facility][vessel][oil]['number_of_transfers'] = df.loc[        
            (df.Receiver == facility) &
            (df.TransferType == 'Cargo') &
            (df.DelivererTypeDescription.isin(type_description)) &
            (df.Product.isin(oil_classification[oil]))
        ].shape[0]

In [11]:
# ~~~  EXPORTS ~~~

# the following lines select all cargo entries registered as received by `tank ships` 
# by the designated marine terminal for the type of oil listed in our 
# `oil_classification` as the DOE oil type represented by our oil categories
for facility in facility_names:
    for oil in oil_types:
        
        # Add up net oil quantity transferred by oil_types
        origin[facility][vessel][oil]['total_gallons'] = df.loc[        
            (df.Deliverer == facility) &
            (df.TransferType == 'Cargo') &
            (df.ReceiverTypeDescription.isin(type_description)) &
            (df.Product.isin(oil_classification[oil])),
            'TransferQtyInGallon'
        ].sum(axis=0).item()
        
        # Calculate the fraction of total oil transfer represented 
        # by this oil type
        if tanker_export_total[facility] != 0:
            origin[facility][vessel][oil]['fraction_of_total'] = decimal_divide(
                origin[facility][vessel][oil]['total_gallons'], 
                tanker_export_total[facility],
                precision
            )
        
        # Catalogue the number of transfers 
        origin[facility][vessel][oil]['number_of_transfers'] = df.loc[        
            (df.Deliverer == facility) &
            (df.TransferType == 'Cargo') &
            (df.ReceiverTypeDescription.isin(type_description)) &
            (df.Product.isin(oil_classification[oil]))
        ].shape[0]

### ATB imports and exports

In [12]:
vessel = 'atb'
type_description = ['TANK BARGE','TUGBOAT']

In [13]:
# ~~~  IMPORTS ~~~

# the following lines select all cargo entries registered as delivered by `tank barge` 
# or `TUGBOAT` (with either `ITB` or `ATB` in the vessel name) to 
# the designated marine terminal for the type of oil listed in our 
# `oil_classification` as the DOE oil type represented by our oil categories
for facility in facility_names:
    for oil in oil_types:
        
        # Add up net oil quantity transferred by oil_types
        destination[facility][vessel][oil]['total_gallons'] = df.loc[        
            (df.Receiver == facility) &
            (df.TransferType == 'Cargo') &
            (df.DelivererTypeDescription.isin(type_description)) &
            ('ATB' in df.Deliverer) | ('ITB' in df.Deliverer) &
            (df.Product.isin(oil_classification[oil])),
            'TransferQtyInGallon'
        ].sum(axis=0).item()
        
        # Calculate the fraction of total oil transfer represented 
        # by this oil type
        if atb_import_total[facility] != 0:
            destination[facility][vessel][oil]['fraction_of_total'] = decimal_divide(
                destination[facility][vessel][oil]['total_gallons'], 
                atb_import_total[facility],
                precision
            )
        
        # Catalogue the number of transfers 
        destination[facility][vessel][oil]['number_of_transfers'] = df.loc[        
            (df.Receiver == facility) &
            (df.TransferType == 'Cargo') &
            (df.DelivererTypeDescription.isin(type_description)) &
            ('ATB' in df.Deliverer) | ('ITB' in df.Deliverer) &
            (df.Product.isin(oil_classification[oil])),
        ].shape[0]

In [14]:
# ~~~  EXPORT ~~~

# the following lines select all cargo entries registered as received by `tank barge` 
# or `TUGBOAT` (with either `ITB` or `ATB` in the vessel name) from 
# the designated marine terminal for the type of oil listed in our 
# `oil_classification` as the DOE oil type represented by our oil categories
for facility in facility_names:
    for oil in oil_types:
        
        # Add up net oil quantity transferred by oil_types
        origin[facility][vessel][oil]['total_gallons'] = df.loc[        
            (df.Deliverer == facility) &
            (df.TransferType == 'Cargo') &
            (df.ReceiverTypeDescription.isin(type_description)) &
            ('ATB' in df.Receiver) | ('ITB' in df.Receiver) &
            (df.Product.isin(oil_classification[oil])),
            'TransferQtyInGallon'
        ].sum(axis=0).item()
        
        # Calculate the fraction of total oil transfer represented 
        # by this oil type
        if atb_export_total[facility] != 0:
            origin[facility][vessel][oil]['fraction_of_total'] = decimal_divide(
                destination[facility][vessel][oil]['total_gallons'], 
                atb_export_total[facility],
                precision
            )
        
        # Catalogue the number of transfers 
        origin[facility][vessel][oil]['number_of_transfers'] = df.loc[        
            (df.Deliverer == facility) &
            (df.TransferType == 'Cargo') &
            (df.ReceiverTypeDescription.isin(type_description)) &
            ('ATB' in df.Receiver) | ('ITB' in df.Receiver) &
            (df.Product.isin(oil_classification[oil])),
        ].shape[0]

### Tank barge imports and exports

In [15]:
vessel = 'barge'
type_description = ['TANK BARGE','TUGBOAT']

In [16]:
# ~~~  IMPORTS ~~~

# the following lines select all cargo entries registered as delivered by `tank barge` 
# or `TUGBOAT` (with neither `ITB` nor `ATB` in the vessel name) to 
# the designated marine terminal for the type of oil listed in our 
# `oil_classification` as the DOE oil type represented by our oil categories
for facility in facility_names:
    for oil in oil_types:
        
        # Add up net oil quantity transferred by oil_types
        destination[facility][vessel][oil]['total_gallons'] = df.loc[        
            (df.Receiver == facility) &
            (df.TransferType == 'Cargo') &
            (df.DelivererTypeDescription.isin(type_description)) &
            ('ATB' not in df.Deliverer) & ('ITB' not in df.Deliverer) &
            (df.Product.isin(oil_classification[oil])),
            'TransferQtyInGallon'
        ].sum(axis=0).item()
        
        # Calculate the fraction of total oil transfer represented 
        # by this oil type
        if barge_import_total[facility] != 0:
            destination[facility][vessel][oil]['fraction_of_total'] = decimal_divide(
                destination[facility][vessel][oil]['total_gallons'], 
                barge_import_total[facility],
                precision
            )
        
        # Catalogue the number of transfers 
        destination[facility][vessel][oil]['number_of_transfers'] = df.loc[        
            (df.Receiver == facility) &
            (df.TransferType == 'Cargo') &
            (df.DelivererTypeDescription.isin(type_description)) &
            ('ATB' not in df.Deliverer) & ('ITB' not in df.Deliverer) &
            (df.Product.isin(oil_classification[oil])),
        ].shape[0]

In [17]:
# ~~~  EXPORT ~~~

# the following lines select all cargo entries registered as received by `tank barge` 
# or `TUGBOAT` (with neither `ITB` nor `ATB` in the vessel name) from 
# the designated marine terminal for the type of oil listed in our 
# `oil_classification` as the DOE oil type represented by our oil categories
for facility in facility_names:
    for oil in oil_types:
        
        # Add up net oil quantity transferred by oil_types
        origin[facility][vessel][oil]['total_gallons'] = df.loc[        
            (df.Deliverer == facility) &
            (df.TransferType == 'Cargo') &
            (df.ReceiverTypeDescription.isin(type_description)) &
            ('ATB' not in df.Receiver) & ('ITB' not in df.Receiver) &
            (df.Product.isin(oil_classification[oil])),
            'TransferQtyInGallon'
        ].sum(axis=0).item()
        
        # Calculate the fraction of total oil transfer represented 
        # by this oil type
        if barge_export_total[facility] != 0:
            origin[facility][vessel][oil]['fraction_of_total'] = decimal_divide(
                origin[facility][vessel][oil]['total_gallons'], 
                barge_export_total[facility],
                precision
            )
        
        # Catalogue the number of transfers 
        origin[facility][vessel][oil]['number_of_transfers'] = df.loc[        
            (df.Deliverer == facility) &
            (df.TransferType == 'Cargo') &
            (df.ReceiverTypeDescription.isin(type_description)) &
            ('ATB' not in df.Receiver) & ('ITB' not in df.Receiver) &
            (df.Product.isin(oil_classification[oil])),
        ].shape[0]

## PROBLEM: Can't get safe_dump to work

In [18]:
with open(r'./WA_origin.yaml', 'w') as file:
    documents = yaml.dump(origin, file)
with open(r'./WA_destination.yaml', 'w') as file:
    documents = yaml.dump(destination, file)

In [19]:
##  QAQC
print('Destination')
for facility in facility_names:
    for vessel in vessel_types:        
        total = sum([destination[facility][vessel][oil]['fraction_of_total'] for oil in oil_types])    
        if (total != 1) & (total != 0):
            print(f'{facility}, {vessel}, {total}')

print('Origin')
for facility in facility_names:
    for vessel in vessel_types:        
        total = sum([origin[facility][vessel][oil]['fraction_of_total'] for oil in oil_types])    
        if (total != 1) & (total != 0):
            print(f'{facility}, {vessel}, {total}')



Destination
BP Cherry Point Refinery, barge, 1.01
Tidewater Snake River Terminal, barge, 1.01
Tesoro Port Angeles Terminal, barge, 0.99
U.S. Oil & Refining, barge, 0.99
Kinder Morgan Liquids Terminal - Harbor Island, barge, 1.01
Origin
BP Cherry Point Refinery, tanker, 0.9999999999999999
Shell Puget Sound Refinery, barge, 1.01
SeaPort Sound Terminal, barge, 1.01
Andeavor Anacortes Refinery (formerly Tesoro), tanker, 0.99
Kinder Morgan Liquids Terminal - Harbor Island, barge, 1.01


---
# Everything below this point is garbage that will be thrown out
---

In [20]:
vessel = 'tanker'
facility = 'BP Cherry Point Refinery'
oil = 'akns'
# add up oil by oil_types
destination[facility][vessel][oil]['total_gallons'] = df.loc[        
    (df.Receiver == facility) &
    (df.TransferType == 'Cargo') &
    (df.DelivererTypeDescription == 'TANK SHIP') &
    (df.Product.isin(oil_classification[oil])),
    'TransferQtyInGallon'
].sum(axis=0)
destination[facility][vessel][oil]['fraction_of_total'] = [
    destination[facility][vessel][oil]['total_gallons'] / 
    tanker_import_total[facility]
]
destination[facility][vessel][oil]['fraction_of_total']

[0.8371278273033612]

In [21]:
destination[facility][vessel][oil]['number_of_transfers'] = df.loc[        
    (df.Receiver == facility) &
    (df.TransferType == 'Cargo') &
    (df.DelivererTypeDescription == 'TANK SHIP') &
    (df.Product.isin(oil_classification[oil]))
].shape[0]
destination[facility][vessel][oil]['number_of_transfers']

103

In [22]:
df.loc[
    (df.Receiver.isin(facility_names)) &
    (df.TransferType == 'Cargo') &
    (df.DelivererTypeDescription.isin(['TANK SHIP'])), 
    'TransferQtyInGallon'
].sum(axis=0)

4412516580

## Create dictionary for oil information

In [23]:
[nrows,ncols] = df.shape

# Create a destination dictionary
destination = {}
destination['names'] = facility_names.copy() 
destination['names'].remove('US Oil Tacoma ') # keeping 'U.S. Oil & Refining'
destination['names'].remove('TLP')           # 'TLP Management Services LLC (TMS)'

# Create an origin dictionary
origin = {}
origin['names'] = facility_names.copy()  #####  REMOVE TLP AND US OIL TACOMA FROM THIS LIST  ####
origin['names'].remove('US Oil Tacoma ') # keeping 'U.S. Oil & Refining'
origin['names'].remove('TLP')           # 'TLP Management Services LLC (TMS)'

for rows in range(nrows):
  
    # Create dictionary structure for destination
    if df.Receiver[rows] in facility_names:
        
        # create dictionary labels for each location (first two are special cases to catch bad data entry)       
        if (df.Receiver[rows] == 'TLP' or 
             df.Receiver[rows] == 'TLP Management Services LLC (TMS)'
           ):
            label = 'TLP Management Services LLC (TMS)'
        elif df.Receiver[rows] == 'U.S. Oil & Refining' or 'US Oil' in df.Receiver[rows]:
            label = 'U.S. Oil & Refining'
        else: # for the entries that are well-behaved!
            label = f'{df.Receiver[rows]}'
        
        # create dictionaries for each location     
        destination[label] = {}

        for vessel in vessel_types:
            destination[label][vessel] = {}
            for oil in oil_types:
                destination[label][vessel][oil] = {}
                for data in data_type:
                    destination[label][vessel][oil][f'{data}'] = 0

 # Create dictionary structure for origin
    if df.Deliverer[rows] in facility_names:
        
        # create dictionary labels for each location (first two are special cases to catch bad data entry)       
        if (df.Deliverer[rows] == 'TLP' or 
            df.Deliverer[rows] == 'TLP Management Services LLC (TMS)'
           ):
            label = 'TLP Management Services LLC (TMS)'      
        elif (df.Deliverer[rows] == 'U.S. Oil & Refining' or 
              'US Oil' in df.Deliverer[rows]
             ):
            label = 'U.S. Oil & Refining'
        else: # for the entries that are well-behaved!
            label = f'{df.Deliverer[rows]}'
        
        # create dictionaries for each location     
        origin[label] = {}
        
        for vessel in vessel_types:
            origin[label][vessel] = {}
            for oil in oil_types:
                origin[label][vessel][oil] = {}
                for data in data_type:
                    origin[label][vessel][oil][f'{data}'] = 0


ValueError: list.remove(x): x not in list

In [ ]:
destination['names'] 

## Loop through data again and catalog oil information

In [ ]:
## Loop through data again and catalog oil information
unregulated_count=1
unregulated_gallons_in = 0
unregulated_gallons_out = 0
####  NOTE: I USE ROWS ABOVE AND ROW HERE.  THAT'S CRAZY MAKING.  I PLAN TO FIX. 
for row in range(nrows):
    
    # Catalogue oil imports (destination traffic)
    if (df.Receiver[row] in facility_names and 
        df.TransferType[row] == 'Cargo'
       ):        
        if (df.Receiver[row] == 'TLP' or 
            df.Receiver[row] == 'TLP Management Services LLC (TMS)'
           ):            
            label = 'TLP Management Services LLC (TMS)'            
        elif (df.Receiver[row] == 'U.S. Oil & Refining' or 
              'US Oil' in df.Receiver[row]
             ):            
            label = 'U.S. Oil & Refining'            
        else: # for the entries that are well-behaved!
            label = f'{df.Receiver[row]}'

        # Now ensure that the deliverer is a tank, barge or atb
        if df.DelivererTypeDescription[row] == 'TANK SHIP':

            # Determine oil-type in transfer 
            if 'CRUDE' in df.Product[row]:
                destination[label]['tanker']['akns']['total_gallons'] += df.TransferQtyInGallon[row].item()
                destination[label]['tanker']['akns']['number_of_transfers'] += 1
                
            elif df.Product[row] == 'BUNKER OIL/HFO':
                destination[label]['tanker']['bunker']['total_gallons'] += df.TransferQtyInGallon[row].item()
                destination[label]['tanker']['bunker']['number_of_transfers'] += 1

            elif df.Product[row] == 'GASOLINE':
                destination[label]['tanker']['gas']['total_gallons'] += df.TransferQtyInGallon[row].item()
                destination[label]['tanker']['gas']['number_of_transfers'] += 1

            elif df.Product[row] == 'JET oil/KEROSENE':
                destination[label]['tanker']['jet']['total_gallons'] += df.TransferQtyInGallon[row].item()
                destination[label]['tanker']['jet']['number_of_transfers'] += 1

            elif 'DIESEL' in df.Product[row]: 
                # inclusive of BIODIESEL, DIESEL/MARINE GAS OIL, and DIESEL LOW SULPHUR (ULSD)
                destination[label]['tanker']['diesel']['total_gallons'] += df.TransferQtyInGallon[row].item()
                destination[label]['tanker']['diesel']['number_of_transfers'] += 1
            elif 'BITUMEN OIL (DIL OR SYN)' in df.Product[row]: 
                destination[label]['tanker']['dilbit']['total_gallons'] += df.TransferQtyInGallon[row].item()
                destination[label]['tanker']['dilbit']['number_of_transfers'] += 1
            else:
                destination[label]['tanker']['other']['total_gallons'] += df.TransferQtyInGallon[row].item()
                destination[label]['tanker']['other']['number_of_transfers'] += 1

        # ATBs
        elif ( df.DelivererTypeDescription[row] == 'TANK BARGE' or 
               df.DelivererTypeDescription[row] == 'TUGBOAT' and 
              'ATB' in df.Deliverer[row] or 
              'ITB' in df.Deliverer[row] ):           
            
             # Determine oil-type in transfer 
            if 'CRUDE' in df.Product[row]:
                destination[label]['atb']['akns']['total_gallons'] += df.TransferQtyInGallon[row].item()
                destination[label]['atb']['akns']['number_of_transfers'] += 1
                
            elif df.Product[row] == 'BUNKER OIL/HFO':
                destination[label]['atb']['bunker']['total_gallons'] += df.TransferQtyInGallon[row].item()
                destination[label]['atb']['bunker']['number_of_transfers'] += 1

            elif df.Product[row] == 'GASOLINE':
                destination[label]['atb']['gas']['total_gallons'] += df.TransferQtyInGallon[row].item()
                destination[label]['atb']['gas']['number_of_transfers'] += 1

            elif df.Product[row] == 'JET oil/KEROSENE':
                destination[label]['atb']['jet']['total_gallons'] += df.TransferQtyInGallon[row].item()
                destination[label]['atb']['jet']['number_of_transfers'] += 1
                
            elif 'DIESEL' in df.Product[row]: 
                # inclusive if BIODIESEL, DIESEL/MARINE GAS OIl, and DIESEL LOW SULPHUR (ULSD)
                destination[label]['atb']['diesel']['total_gallons'] += df.TransferQtyInGallon[row].item()
                destination[label]['atb']['diesel']['number_of_transfers'] += 1
                
            elif 'BITUMEN OIL (DIL OR SYN)' in df.Product[row]: 
                destination[label]['atb']['dilbit']['total_gallons'] += df.TransferQtyInGallon[row].item()
                destination[label]['atb']['dilbit']['number_of_transfers'] += 1
                
            else:
                destination[label]['atb']['other']['total_gallons'] += df.TransferQtyInGallon[row].item()
                destination[label]['atb']['other']['number_of_transfers'] += 1
        
        # Tank-barges
        elif ( df.DelivererTypeDescription[row] == 'TANK BARGE' or 
               df.DelivererTypeDescription[row] == 'TUGBOAT' and 
              'ATB' not in df.Deliverer[row] and 
              'ITB' not in df.Deliverer[row] ):
            
            # Determine oil-type in transfer 
            if 'CRUDE' in df.Product[row]:
                destination[label]['barge']['akns']['total_gallons'] += df.TransferQtyInGallon[row].item()
                destination[label]['barge']['akns']['number_of_transfers'] += 1
                
            elif df.Product[row] == 'BUNKER OIL/HFO':
                destination[label]['barge']['bunker']['total_gallons'] += df.TransferQtyInGallon[row].item()
                destination[label]['barge']['bunker']['number_of_transfers'] += 1

            elif df.Product[row] == 'GASOLINE':
                destination[label]['barge']['gas']['total_gallons'] += df.TransferQtyInGallon[row].item()
                destination[label]['barge']['gas']['number_of_transfers'] += 1

            elif df.Product[row] == 'JET oil/KEROSENE':
                destination[label]['barge']['jet']['total_gallons'] += df.TransferQtyInGallon[row].item()
                destination[label]['barge']['jet']['number_of_transfers'] += 1

            elif 'DIESEL' in df.Product[row]: 
                # inclusive if BIODIESEL, DIESEL/MARINE GAS OIl, and DIESEL LOW SULPHUR (ULSD)
                destination[label]['barge']['diesel']['total_gallons'] += df.TransferQtyInGallon[row].item()
                destination[label]['barge']['diesel']['number_of_transfers'] += 1
            
            elif 'BITUMEN OIL (DIL OR SYN)' in df.Product[row]: 
                destination[label]['barge']['dilbit']['total_gallons'] += df.TransferQtyInGallon[row].item()
                destination[label]['barge']['dilbit']['number_of_transfers'] += 1

            else:
                destination[label]['barge']['other']['total_gallons'] += df.TransferQtyInGallon[row].item()
                destination[label]['barge']['other']['number_of_transfers'] += 1
            
            # find tugs transporting oil from Westridge
            if label=='U.S. Oil & Refining':
                tugs_from_westridge = tugs_from_westridge.append( df.iloc[row] )
                
        else:
            unregulated_count += 1
            unregulated_gallons_in += df.TransferQtyInGallon[row].item()
            print(f'[{row}]: Cargo transfer of {df.TransferQtyInGallon[row].item():4.2e} gallons of {df.Product[row]} from {df.DelivererTypeDescription[row]} to {df.Receiver[row]} ')
            
   # Catalogue oil exports (origin traffic)
    if (df.Deliverer[row] in facility_names and 
        df.TransferType[row] == 'Cargo'
       ):
        
        if (df.Deliverer[row] == 'TLP' or 
            df.Deliverer[row] == 'TLP Management Services LLC (TMS)'
           ):
            label = 'TLP Management Services LLC (TMS)'
        elif (df.Deliverer[row] == 'U.S. Oil & Refining' or 
              'US Oil' in df.Deliverer[row]
             ):
            label = 'U.S. Oil & Refining'
        else: # for the entries that are well-behaved!
            label = f'{df.Deliverer[row]}'

        # Now ensure that the deliverer is a tank, barge or atb
        if df.ReceiverTypeDescription[row] == 'TANK SHIP':

            # Determine oil-type in transfer 
            if 'CRUDE' in df.Product[row]:
                origin[label]['tanker']['akns']['total_gallons'] += df.TransferQtyInGallon[row].item()
                origin[label]['tanker']['akns']['number_of_transfers'] += 1
                
            elif df.Product[row] == 'BUNKER OIL/HFO':
                origin[label]['tanker']['bunker']['total_gallons'] += df.TransferQtyInGallon[row].item()
                origin[label]['tanker']['bunker']['number_of_transfers'] += 1

            elif df.Product[row] == 'GASOLINE':
                origin[label]['tanker']['gas']['total_gallons'] += df.TransferQtyInGallon[row].item()
                origin[label]['tanker']['gas']['number_of_transfers'] += 1

            elif df.Product[row] == 'JET oil/KEROSENE':
                origin[label]['tanker']['jet']['total_gallons'] += df.TransferQtyInGallon[row].item()
                origin[label]['tanker']['jet']['number_of_transfers'] += 1

            elif 'DIESEL' in df.Product[row]: 
                # inclusive if BIODIESEL, DIESEL/MARINE GAS OIl, and DIESEL LOW SULPHUR (ULSD)
                origin[label]['tanker']['diesel']['total_gallons'] += df.TransferQtyInGallon[row].item()
                origin[label]['tanker']['diesel']['number_of_transfers'] += 1
             
            elif 'BITUMEN OIL (DIL OR SYN)' in df.Product[row]: 
                origin[label]['tanker']['dilbit']['total_gallons'] += df.TransferQtyInGallon[row].item()
                origin[label]['tanker']['dilbit']['number_of_transfers'] += 1

            else:
                origin[label]['tanker']['other']['total_gallons'] += df.TransferQtyInGallon[row].item()
                origin[label]['tanker']['other']['number_of_transfers'] += 1
        
        # ATBs
        elif ( df.ReceiverTypeDescription[row] == 'TANK BARGE' or 
               df.ReceiverTypeDescription[row] == 'TUGBOAT' and 
              'ATB' in df.Receiver[row] or 
              'ITB' in df.Receiver[row] ):
             
             # Determine oil-type in transfer 
            if 'CRUDE' in df.Product[row]:
                origin[label]['atb']['akns']['total_gallons'] += df.TransferQtyInGallon[row].item()
                origin[label]['atb']['akns']['number_of_transfers'] += 1
                
            elif df.Product[row] == 'BUNKER OIL/HFO':
                origin[label]['atb']['bunker']['total_gallons'] += df.TransferQtyInGallon[row].item()
                origin[label]['atb']['bunker']['number_of_transfers'] += 1

            elif df.Product[row] == 'GASOLINE':
                origin[label]['atb']['gas']['total_gallons'] += df.TransferQtyInGallon[row].item()
                origin[label]['atb']['gas']['number_of_transfers'] += 1

            elif df.Product[row] == 'JET oil/KEROSENE':
                origin[label]['atb']['jet']['total_gallons'] += df.TransferQtyInGallon[row].item()
                origin[label]['atb']['jet']['number_of_transfers'] += 1

            elif 'DIESEL' in df.Product[row]: 
                # inclusive if BIODIESEL, DIESEL/MARINE GAS OIL, and DIESEL LOW SULPHUR (ULSD)
                origin[label]['atb']['diesel']['total_gallons'] += df.TransferQtyInGallon[row].item()
                origin[label]['atb']['diesel']['number_of_transfers'] += 1
                
            elif 'BITUMEN OIL (DIL OR SYN)' in df.Product[row]: 
                origin[label]['atb']['dilbit']['total_gallons'] += df.TransferQtyInGallon[row].item()
                origin[label]['atb']['dilbit']['number_of_transfers'] += 1

            else:
                origin[label]['atb']['other']['total_gallons'] += df.TransferQtyInGallon[row].item()
                origin[label]['atb']['other']['number_of_transfers'] += 1
        
        # Tank-barges
        elif ( df.ReceiverTypeDescription[row] == 'TANK BARGE' or 
               df.ReceiverTypeDescription[row] == 'TUGBOAT' and 
              'ATB' not in df.Receiver[row] and 
              'ITB' not in df.Receiver[row] ):
            
            # Determine oil-type in transfer 
            if 'CRUDE' in df.Product[row]:
                origin[label]['barge']['akns']['total_gallons'] += df.TransferQtyInGallon[row].item()
                origin[label]['barge']['akns']['number_of_transfers'] += 1
                
            elif df.Product[row] == 'BUNKER OIL/HFO':
                origin[label]['barge']['bunker']['total_gallons'] += df.TransferQtyInGallon[row].item()
                origin[label]['barge']['bunker']['number_of_transfers'] += 1

            elif df.Product[row] == 'GASOLINE':
                origin[label]['barge']['gas']['total_gallons'] += df.TransferQtyInGallon[row].item()
                origin[label]['barge']['gas']['number_of_transfers'] += 1

            elif df.Product[row] == 'JET oil/KEROSENE':
                origin[label]['barge']['jet']['total_gallons'] += df.TransferQtyInGallon[row].item()
                origin[label]['barge']['jet']['number_of_transfers'] += 1

            elif 'DIESEL' in df.Product[row]: 
                # inclusive if BIODIESEL, DIESEL/MARINE GAS OIl, and DIESEL LOW SULPHUR (ULSD)
                origin[label]['barge']['diesel']['total_gallons'] += df.TransferQtyInGallon[row].item()
                origin[label]['barge']['diesel']['number_of_transfers'] += 1
            
            elif 'BITUMEN OIL (DIL OR SYN)' in df.Product[row]: 
                origin[label]['barge']['dilbit']['total_gallons'] += df.TransferQtyInGallon[row].item()
                origin[label]['barge']['dilbit']['number_of_transfers'] += 1

            else:
                origin[label]['barge']['other']['total_gallons'] += df.TransferQtyInGallon[row].item()
                origin[label]['barge']['other']['number_of_transfers'] += 1
            
        else:
            unregulated_count += 1
            unregulated_gallons_out += df.TransferQtyInGallon[row].item()
            print(f'[{row}]: Cargo transfer of {df.TransferQtyInGallon[row].item():4.2e} gallons of {df.Product[row]} from {df.Deliverer[row]} to {df.ReceiverTypeDescription[row]} ')
            

In [ ]:
unregulated_count

In [ ]:
unregulated_gallons_out

In [ ]:
unregulated_gallons_in

###  Set zero values to locations where no oil transfers are recorded for cargo

In [ ]:
for facility in origin['names']:
    if facility not in origin:
        
        print(f'Origin: Adding null values to {facility}')
        
        # create dictionaries for each location     
        origin[facility] = {}
        
        for vessel in vessel_types:
            origin[facility][vessel] = {}
            for oil in oil_types:
                origin[facility][vessel][oil] = {}
                for data in data_type:
                    origin[facility][vessel][oil][f'{data}'] = 0

for facility in destination['names']:
    if facility not in destination:
        
        print(f'Origin: Adding null values to {facility}')
        
        # create dictionaries for each location     
        destination[facility] = {}
        
        for vessel in vessel_types:
            destination[facility][vessel] = {}
            for oil in oil_types:
                destination[facility][vessel][oil] = {}
                for data in data_type:
                    destination[facility][vessel][oil][f'{data}'] = 0


In [ ]:
## double check to make sure this is right.  

no_cargo_terminals = ['Shell Oil LP Seattle Distribution Terminal',
                     'Phillips 66 Tacoma Terminal',
                     'Naval Air Station Whidbey Island (NASWI)',
                     'REG Grays Harbor, LLC']
origin_check = {}
destination_check = {}
    
for terminal in no_cargo_terminals:
    df_destination = df[df['Receiver'] == terminal]
    df_origin = df[df['Deliverer'] == terminal]
    destination_check[terminal] = sum(
        df_destination[df_destination['TransferType'] == 'Cargo'].TransferQtyInGallon
    )
    origin_check[terminal] = sum(
        df_origin[df_origin['TransferType'] == 'Cargo'].TransferQtyInGallon
    )
    

In [ ]:
destination_check

In [ ]:
origin_check

### Loop through each location, vessel type and oil type and determine percent oil for each oil type

In [ ]:
misbehaving_facility = []
misbehaving_vessel = []
for facility in origin['names']:
    for vessel in vessel_types:
        
        # first get the total transfers over all oil types
        total_origin = sum(
            np.around(
                [origin[facility][vessel][count_me_in]['total_gallons'] for count_me_in in oil_types]
            )
        )
        
        total_destination = sum(
            np.around(
                [destination[facility][vessel][count_me_in]['total_gallons'] for count_me_in in oil_types]
        
            )
        )
    
        sum_origin = 0.0
        sum_destination = 0.0
        for oil in oil_types:
            # next calculate percentages
            if total_origin != 0:
                origin[facility][vessel][oil]['fraction_of_total'] = decimal_divide(
                    origin[facility][vessel][oil]['total_gallons'], 
                    total_origin,
                    precision
                )
                sum_origin += origin[facility][vessel][oil]['fraction_of_total']
                    
#                 origin[facility][vessel][oil]['fraction_of_total'] =  decimal_divide(
#                     origin[facility][vessel][oil]['total_gallons'],
#                     total_origin,
#                     10
#                 )
                
            if total_destination != 0:
                destination[facility][vessel][oil]['fraction_of_total'] = decimal_divide(
                    destination[facility][vessel][oil]['total_gallons'],
                    total_destination,
                    precision
                )
                sum_destination += destination[facility][vessel][oil]['fraction_of_total']
        
        if sum_origin != 0.0 and sum_origin != 1.0:
            misbehaving_facility.append(facility)
            misbehaving_vessel.append(vessel) 
            
            # manually force sum of weights to zero by adding or removing from "other"
            shortfall  = 1-sum_origin
            if shortfall != 1.0: # don't add oil transfers where none exist!
                if shortfall > 0.0: 
                    origin[facility][vessel]['other']['fraction_of_total'] = (
                        origin[facility][vessel]['other']['fraction_of_total'] + shortfall
                    )        
                    
        print(f'Sum of origin oil weights: {sum_origin}')
        print(f'Sum of destination oil weights: {sum_destination}')

In [ ]:
misbehaving_facility

In [ ]:
misbehaving_vessel

In [ ]:
# # Fix 'other' fraction of oil to be 1 - sum of other oil types
# for facility in origin['names']:
#     for vessel in vessel_types:
#         # Add all "non-other" oil-types
#         origin_sum = sum([origin[facility][vessel][count_me_in]['fraction_of_total'] for count_me_in in oil_types[0:6]])
#         destination_sum = sum([destination[facility][vessel][count_me_in]['fraction_of_total'] for count_me_in in oil_types[0:6]])
        
#         # force sum of fraction_of_total to one by pooling "leftovers" in "other"       
#         if origin_sum > 1:
#             gas_correction = 1 - origin_sum
#             # gas works in this case but isn't a general fix
#             origin[facility][vessel]['gas']['fraction_of_total'] = (
#                 origin[facility][vessel]['gas']['fraction_of_total'] + gas_correction
#             )
#             print(f'{facility}, {vessel}, origin gas correction: {gas_correction}')
            
#         elif origin_sum < 1:
#             origin[facility][vessel]['other']['fraction_of_total'] =  (
#             1 - origin_sum
#         )
            
#         if destination_sum > 1:
#             gas_correction = 1 - destination_sum
#             origin[facility][vessel]['gas']['fraction_of_total'] = (
#                 origin[facility][vessel]['gas']['fraction_of_total'] + gas_correction
#             )
#             origin[facility][vessel]['other']['fraction_of_total'] = 0
#             print(f'{facility}, {vessel}, destination gas correction: {gas_correction}')          
        
#         elif destination_sum < 1:
#             destination[facility][vessel]['other']['fraction_of_total'] =  (
#             1 - destination_sum
#         )   

In [ ]:
for oil in oil_types:
    string = str(destination['U.S. Oil & Refining']['atb'][oil]['total_gallons'])
    print(string)
    
sum([destination['U.S. Oil & Refining']['atb'][oil]['fraction_of_total'] for oil in oil_types])    

In [ ]:
for oil in oil_types:
    print(origin[facility][vessel][oil]['fraction_of_total'])

In [ ]:
for oil in oil_types:
    string = str(destination['U.S. Oil & Refining']['atb'][oil]['fraction_of_total'])
    print(string)
    
sum([destination['U.S. Oil & Refining']['atb'][oil]['fraction_of_total'] for oil in oil_types])    

In [ ]:
for facility in origin['names']:
    for vessel in vessel_types:
        sum_check_out = [origin[facility][vessel][oil]['fraction_of_total'] for oil in oil_types]
        sum_check_in  = [destination[facility][vessel][oil]['fraction_of_total'] for oil in oil_types]
        if sum(sum_check_out) != 1:
            # add oil to "other" category to fix sum at 1
            shortfall  = 1-sum(sum_check_out)
            if shortfall != 1: # don't add oil transfers where none exist!
                origin[facility][vessel]['other']['fraction_of_total'] = (
                    origin[facility][vessel]['other']['fraction_of_total'] + shortfall
                )
                print(f'origin {vessel}, {facility} SHORTFALL:{shortfall}')

            # double check
            sum_check_out = [origin[facility][vessel][oil]['fraction_of_total'] for oil in oil_types]
            if sum(sum_check_out) != 1 and sum(sum_check_out) !=0 :
                print(f'NOT THERE YET!: origin {vessel}, {facility}:{sum(sum_check_out)}')
        
        elif sum(sum_check_in) != 1:
            print(f'destination {vessel}, {facility}:{sum(sum_check_in)}')
            
            # add oil to "other" category to fix sum at 1
            shortfall  = 1-sum(sum_check_in)
            if shortfall != 1: # don't add oil transfers where none exist!
                destination[facility][vessel]['other']['fraction_of_total'] = (
                    destination[facility][vessel]['other']['fraction_of_total'] + shortfall
                )
                print(f'origin {vessel}, {facility} SHORTFALL:{shortfall}')

            # double check
            sum_check_in = [origin[facility][vessel][oil]['fraction_of_total'] for oil in oil_types]
            if sum(sum_check_in) != 1 and sum(sum_check_in) !=0 :
                print(f'NOT THERE YET!: origin {vessel}, {facility}:{sum(sum_check_in)}')

In [ ]:
sum([destination['U.S. Oil & Refining']['atb'][oil]['fraction_of_total'] for oil in oil_types])

In [ ]:
for oil in oil_types:
    string = str(destination['U.S. Oil & Refining']['atb'][oil]['fraction_of_total'])
    print(string)
    destination['U.S. Oil & Refining']['atb'][oil]['fraction_of_total'] = float(string)
    
sum([destination['U.S. Oil & Refining']['atb'][oil]['fraction_of_total'] for oil in oil_types])    

In [ ]:
origin


In [ ]:
with open(r'./WA_origin.yaml', 'w') as file:
    documents = yaml.safe_dump(origin, file)
with open(r'./WA_destination.yaml', 'w') as file:
    documents = yaml.safe_dump(destination, file)

### plot up results for WA origin and destination by vessel type

In [ ]:
label = 'barge'
plot_origin = {}
facility = 'BP Cherry Point Refinery'
vessel_types = ['tanker','atb','barge']
for vessel in vessel_types:
    plot_origin[vessel] = np.empty(7)
    oil_index = 0
    for oil in oil_types:
        plot_origin[vessel][oil_index]=origin[facility][vessel][oil]['total_gallons']
        oil_index += 1

In [ ]:
# axes are in a two-dimensional array, indexed by [row, col]
plot_array = np.empty(len(oil_types))

# Create a circle for the center of the plot
#my_circle=plt.Circle( (0,0), 0.7, color='white')

for i in range(len(destination['names'])):
    fig, axes = plt.subplots(1, len(vessel_types), figsize = (15,5))
    if i == 0:
        fig.suptitle('Washington oil imports',
                    fontsize = 24)
    facility = origin['names'][i]
    plt.text(-.1, 0.5, 
             facility, 
             ha='left',
             fontsize=14,
             wrap=True,
             transform=plt.gcf().transFigure)
    for j in range(len(vessel_types)):
        vessel = vessel_types[j]
        # create array of total oil volume by oil type for each vessel at each facility
        oil_index = 0
        for oil in oil_types:    
            plot_array[oil_index]=destination[facility][vessel][oil]['total_gallons']
            oil_index += 1
        
        axes[j].axis('equal')
        
        if plot_array.sum()>0:
            axes[j].pie(plot_array, wedgeprops=dict(width=0.5), labels = oil_type)
        else:
            axes[j].axis('off')
        axes[j].text(0,0,vessel,ha='center',fontsize=14)
       
          

In [ ]:
# axes are in a two-dimensional array, indexed by [row, col]
plot_array = np.empty(len(oil_types))

# Create a circle for the center of the plot
#my_circle=plt.Circle( (0,0), 0.7, color='white')

for i in range(len(origin['names'])):
    fig, axes = plt.subplots(1, len(vessel_types), figsize = (15,5))
    if i == 0:
        fig.suptitle('Washington oil exports',
                    fontsize = 24)
    facility = origin['names'][i]
    plt.text(-.1, 0.5, 
             facility, 
             ha='left',
             fontsize=14,
             wrap=True,
             transform=plt.gcf().transFigure)
    for j in range(len(vessel_types)):
        vessel = vessel_types[j]
        # create array of total oil volume by oil type for each vessel at each facility
        oil_index = 0
        for oil in oil_types:    
            plot_array[oil_index]=origin[facility][vessel][oil]['total_gallons']
            oil_index += 1
        
        axes[j].axis('equal')
        
        if plot_array.sum()>0:
            axes[j].pie(plot_array, wedgeprops=dict(width=0.5), labels = oil_type)
        else:
            axes[j].axis('off')
        axes[j].text(0,0,vessel,ha='center',fontsize=14)
       
            
